In [122]:
import numpy as np
import matplotlib.pyplot as plt
import pde
import os


# Boundary conditions test

In [123]:
class CustomPDE(pde.PDEBase):
    def __init__(self, boundary_mask):
        self.boundary_mask = boundary_mask

    def evolution_rate(self, state, t=0):
        """ Custom PDE evolution with modified diffusion rate """
        laplacian = state.laplace(bc={"value": 0})  # zero Dirichlet boundary condition
        rate = laplacian.copy()  # Default rate inside the grid
        
        # Reduce the rate of diffusion at grid to a quarter
        rate.data[self.boundary_mask] *= 0.25
        
        return rate


In [124]:
# generate grid
grid_size = [64, 64]
grid = pde.UnitGrid(grid_size)  
state = pde.ScalarField(grid, data=0)

 # insert peaks in the centers
state.insert([10.5, 10.5], 1)
state.insert([32, 10.5], 1)
state.insert([53.3, 10.5], 1)
state.insert([10.5, 32], 1)
state.insert([32, 32], 1) 
state.insert([53.3, 32], 1)
state.insert([10.5, 53.3], 1)
state.insert([32, 53.3], 1)
state.insert([53.3, 53.3], 1)

# Define the mask for grid lines with thickness 1
x, y = grid.cell_coords[..., 0], grid.cell_coords[..., 1]
section_size = grid_size[0] // 3
boundary_mask = (
    ((x % section_size <= 2) | (x % section_size >= section_size - 2)) |
    ((y % section_size <= 2) | (y % section_size >= section_size - 2))
)

eq = CustomPDE(boundary_mask)
storage = pde.MemoryStorage()
result = eq.solve(state, t_range=30, dt=0.1, tracker=storage.tracker(0.1))

In [125]:
# Plot calculated results
# WARNING: This will take a long time to run, uncomment with care
pde.movie(storage, filename="output.mp4", plot_args={"vmin": 0, "vmax": 0.5})

100%|██████████| 300/300 [00:17<00:00, 16.92it/s]


# JIT test

In [126]:
from pde.tools.numba import jit

In [127]:

class KuramotoSivashinskyPDE(pde.PDEBase):

    def __init__(self, diffusivity=1, bc="auto_periodic_neumann", bc_laplace="auto_periodic_neumann"):
        """ initialize the class with a diffusivity and boundary conditions
        for the actual field and its second derivative """
        self.diffusivity = diffusivity
        self.bc = bc
        self.bc_laplace = bc_laplace


    def evolution_rate(self, state, t=0):
        """ numpy implementation of the evolution equation """
        state_lapacian = state.laplace(bc=self.bc)
        state_gradient = state.gradient(bc="auto_periodic_neumann")
        return (- state_lapacian.laplace(bc=self.bc_laplace)
                - state_lapacian
                - 0.5 * self.diffusivity * (state_gradient @ state_gradient))


    def _make_pde_rhs_numba(self, state):
        """ the numba-accelerated evolution equation """
        # make attributes locally available
        diffusivity = self.diffusivity

        # create operators
        laplace_u = state.grid.make_operator("laplace", bc=self.bc)
        gradient_u = state.grid.make_operator("gradient", bc=self.bc)
        laplace2_u = state.grid.make_operator("laplace", bc=self.bc_laplace)
        dot = pde.VectorField(state.grid).make_dot_operator()

        @jit
        def pde_rhs(state_data, t=0):
            """ compiled helper function evaluating right hand side """
            state_lapacian = laplace_u(state_data)
            state_grad = gradient_u(state_data)
            return (- laplace2_u(state_lapacian)
                    - state_lapacian
                    - diffusivity / 2 * dot(state_grad, state_grad))

        return pde_rhs

In [128]:
grid_size = [64, 64]
grid = pde.UnitGrid(grid_size, periodic=True)
state = pde.ScalarField(grid, data=np.random.rand(*grid.shape))

eq = KuramotoSivashinskyPDE(diffusivity=0.1)
storage = pde.MemoryStorage()
result = eq.solve(state, t_range=33, dt=0.01, tracker=storage.tracker(0.1))

In [129]:
# Plot calculated results
# WARNING: This will take a long time to run, uncomment with care

# Set the scale of the colorbar to be fixed
pde.movie(storage, filename="output2.mp4", plot_args={"vmin": -30, "vmax": 30})

  0%|          | 0/331 [00:00<?, ?it/s]

100%|██████████| 331/331 [00:20<00:00, 16.30it/s]
